In [1]:
"""
啟用伺服器基本樣板
"""
# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 
from linebot.exceptions import (
    InvalidSignatureError
)

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageMessage, ImagemapSendMessage, BaseSize,
    URIImagemapAction, ImagemapArea, TemplateSendMessage, URITemplateAction, ButtonsTemplate,
    MessageImagemapAction, MessageTemplateAction, PostbackTemplateAction,
    ConfirmTemplate, PostbackTemplateAction, MessageTemplateAction, DatetimePickerAction,
    CarouselColumn, CarouselTemplate, ImageCarouselTemplate, ImageCarouselColumn, StickerSendMessage
)

# 載入設定檔

import json
secretFileContentJson=json.load(open("./line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")


# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "images", template_folder='./')

from datetime import timedelta
app.config['SEND_FILE_MAX_AGE_DEFAULT'] = timedelta(seconds=10)

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

import logging
eventHandler = logging.FileHandler('event.log')
logger2 = logging.getLogger('chatbot')
logger2.setLevel(logging.DEBUG)
logger2.addHandler(eventHandler)

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)
    logger2.info(body)
    
    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'


In [2]:
'''
製作文字與圖片的教學訊息
'''
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage, PostbackEvent
)

# 消息清單
reply_message_list = [
TextSendMessage(text="歡迎使用我的智能衣櫃!"),
    TextSendMessage(text="點選底下之菜單「開始使用我的智能衣櫃」以加入會員使用完整功能，也可以以文字輸入「關於我」，來獲得使用說明。"),
]

In [3]:
'''
撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 把先前製作好的自定義菜單，與用戶做綁定
3. 回應用戶，歡迎用的文字消息與圖片消息
'''

# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

# 載入requests套件
import requests
import pymysql 

# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("../users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
        
    
    db = pymysql.connect("mysql","root",secretFileContentJson["mysql_pass"],"user_data" )
    cursor = db.cursor()
    cursor.execute('select line_id from line_user' )
    results = cursor.fetchall()
    
    useridlist = []
    for userid in results:
        useridlist.append(userid[0])
    
    if not event.source.user_id in useridlist:
 
    # 將菜單綁定在用戶身上
        linkRichMenuId="richmenu-caab909ce44f2957cee033aeda34188e"
        linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (event.source.user_id, linkRichMenuId)
        linkMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}
        lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
    else:
        linkRichMenuId="richmenu-02e98659ec5f8ebd88f0db2bed101d69"
        linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (event.source.user_id, linkRichMenuId)
        linkMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}
        lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
        event.reply_token,
        reply_message_list
    )

In [4]:
from flask import Flask, request
from flask import render_template
import base64
from pymongo import MongoClient
from bson import binary
import datetime
import pymysql
import random
import re
import time
import jieba
from sklearn.externals import joblib
%run weather_function.ipynb


def word(s):
    return " ".join(jieba.cut(s)).replace("\r", "").replace("\n", "").replace("。", "").replace("，", "")

# 收到使用者資訊後由chatbot回傳資訊或動作
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    
    line_id_mo = vars(user_profile)["user_id"]
    msg = event.message.text
    
    if msg == "功能說明" or msg == "上一頁":
        features = TemplateSendMessage(
            alt_text='功能說明',
            template=ButtonsTemplate(
                title='使用功能說明 第一頁',
                text='請選擇您想了解的功能',
                thumbnail_image_url='https://i.imgur.com/WUFlprb.png',
                actions=[
                    PostbackTemplateAction(
                        label='加入會員',
                        data="加入會員說明"
                    ),
                    PostbackTemplateAction(
                        label='查看個人資料',
                        data="查看個人資料說明"
                    ),
                    PostbackTemplateAction(
                        label='情境輸入',
                        data="情境輸入說明"
                    ),
                    MessageTemplateAction(
                        label='下一頁',
                        text='下一頁'
                    )
                ]
            )
        )
        line_bot_api.reply_message(event.reply_token, features)
    elif msg == "不用謝謝":
        pass
    elif msg == "加入會員":
        pass
    elif msg == "關於我":
        pass
    elif msg == "查看個人資料":
        pass
    elif msg == "情境輸入":
        pass
    elif msg == "開啟衣櫃":
        pass
    elif msg == "我的明星臉":
        pass
    elif msg == "下一頁":
        features2 = TemplateSendMessage(
            alt_text='功能說明',
            template=ButtonsTemplate(
                title='使用功能說明 第二頁',
                text='請選擇您想了解的功能',
                thumbnail_image_url='https://i.imgur.com/WUFlprb.png',
                actions=[
                    PostbackTemplateAction(
                        label='拍照／圖庫上傳',
                        data="拍照／圖庫上傳說明"
                    ),
                    PostbackTemplateAction(
                        label='開啟衣櫃',
                        data="開啟衣櫃說明"
                    ),
                    PostbackTemplateAction(
                        label='我的明星臉',
                        data="我的明星臉說明"
                    ),
                    MessageTemplateAction(
                        label='上一頁',
                        text='上一頁'
                    )
                ]
            )
        )
        line_bot_api.reply_message(event.reply_token, features2)

    elif msg == "進行運動相關的活動，開始推薦":
        import pymysql
        import random
        import redis
        try:
            reco_clo = random.choice(['背心', 'T-shirt'])

            db = pymysql.connect("mysql","root",secretFileContentJson["mysql_pass"],"user_data" )
            cursor = db.cursor()
            cursor.execute('select * from line_user where line_id = '+ '"' + line_id_mo + '"' )
            results = cursor.fetchone()
            usex = results[4]

            client = MongoClient('mongo', 27017)
            db = client.photo_flask
            coll = db.images
            try:
                data = coll.find({"line_id": line_id_mo, 'kinds': reco_clo})
                noteimg = random.randint(0,data.count()-1)
                with open('./images/user_clo/'+ line_id_mo + '.jpg', 'wb') as fn:
                            fn.write(data[noteimg]['imagecontent'])
            except ValueError:
                rmc = ['背心', 'T-shirt']
                rmc.remove(reco_clo)
                data = coll.find({"line_id": line_id_mo, 'kinds': rmc[0]})
                noteimg = random.randint(0,data.count()-1)
                with open('./images/user_clo/'+ line_id_mo + '.jpg', 'wb') as fn:
                            fn.write(data[noteimg]['imagecontent'])
                reco_clo = rmc[0]

            user_add1 = ImageSendMessage(original_content_url=server_url + '/images/user_clo/' + line_id_mo + '.jpg',
                                        preview_image_url=server_url + '/images/user_clo/' + line_id_mo + '.jpg')

            uss = {'男': 'M', '女': 'F'}
            r = redis.Redis(host='redis', port=6379, decode_responses=True)
            rk = r.keys("*_"+reco_clo+"_"+uss[usex]+"*")
            randomlist = random.sample(rk, 5)
            rn, ri, ru = [], [], []
            for clora in randomlist:
                rn.append(r.get(clora).split('___')[0])
                ri.append(r.get(clora).split('___')[1])
                ru.append(r.get(clora).split('___')[2])
            Carousel_templateC = TemplateSendMessage(
                    alt_text='運動相關推薦',
                    template=CarouselTemplate(
                    columns=[
                        CarouselColumn(
                            thumbnail_image_url=ri[0],
                            title=rn[0],
                            text=reco_clo,
                            actions=[
                                URITemplateAction(
                                    label='前往選購',
                                    uri=ru[0]
                                )
                            ],
                        ),
                        CarouselColumn(
                            thumbnail_image_url=ri[1],
                            title=rn[1],
                            text=reco_clo,
                            actions=[
                                URITemplateAction(
                                    label='前往選購',
                                    uri=ru[1]
                                )
                            ]
                        ),
                        CarouselColumn(
                            thumbnail_image_url=ri[2],
                            title=rn[2],
                            text=reco_clo,
                            actions=[
                                URITemplateAction(
                                    label='前往選購',
                                    uri=ru[2]
                                )
                            ]
                        ),
                        CarouselColumn(
                            thumbnail_image_url=ri[3],
                            title=rn[3],
                            text=reco_clo,
                            actions=[
                                URITemplateAction(
                                    label='前往選購',
                                    uri=ru[3]
                                )
                            ]
                        ),
                        CarouselColumn(
                            thumbnail_image_url=ri[4],
                            title=rn[4],
                            text=reco_clo,
                            actions=[
                                URITemplateAction(
                                    label='前往選購',
                                    uri=ru[4]
                                )
                            ]
                        ),
                    ]
                )
                )
            Confirm_templateT = TemplateSendMessage(
                alt_text='行程提醒',
                template=ConfirmTemplate(
                    title='行程提醒',
                    text='是否需要新增行程提醒?',
                    actions=[                              
                        DatetimePickerAction(
                                    label="新增提醒",
                                    data="usertime"+"/"+str(noteimg)+"/"+reco_clo,
                                    mode="datetime",
                                    initial=(datetime.datetime.now() + datetime.timedelta(hours=8)).strftime("%Y-%m-%dT%H:%M"),
                                    min=(datetime.datetime.now() + datetime.timedelta(hours=8)).strftime("%Y-%m-%dT%H:%M"),
                                    max="2038-09-08T23:59"
                                ),
                        MessageTemplateAction(
                            label='暫時不用',
                            text='不用謝謝',

                                )
                            ]
                        )
                    )

            user_profile = line_bot_api.get_profile(event.source.user_id)
            client = MongoClient('mongo', 27017)
            db = client.photo_flask
            coll = db.userevent
            data = coll.find({"line_id": vars(user_profile)["user_id"]}).sort('_id',-1).limit(1)

            try:
                weather_input = date_func(data[0]['event'])
#                 print(weather_input)
                weather_output = return_weather(weather_input)
#                 print(weather_output)
#                 print(weather_output.split('*'))
                weather_final = "桃園市"+"\n"+weather_output.split('*')[0] + weather_output.split('*')[1] +"\n天氣狀態: " \
                                    + weather_output.split('*')[2] +"\n"+ weather_output.split('*')[3] \
                                    +"\n氣溫: "+ weather_output.split('*')[5]+"～" + weather_output.split('*')[4]
#                     print(weather_final)
                line_bot_api.reply_message(event.reply_token, [TextSendMessage(text=weather_final), TextSendMessage(text='推薦您穿著：'+reco_clo), user_add1, Carousel_templateC, Confirm_templateT])
            except:
                line_bot_api.reply_message(event.reply_token, [TextSendMessage(text='推薦您穿著：'+reco_clo), user_add1, Carousel_templateC, Confirm_templateT])

        except TypeError:
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text='沒有使用者資料，請先加入會員哦～'))
        except ValueError:
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text='沒有相關衣服哦～'))

    elif msg == "進行休閒相關的活動，開始推薦":
        import pymysql
        import random
        import redis
        try:
            reco_clo = random.choice(['背心', 'T-shirt', '襯衫'])

            db = pymysql.connect("mysql","root",secretFileContentJson["mysql_pass"],"user_data" )
            cursor = db.cursor()
            cursor.execute('select * from line_user where line_id = '+ '"' + line_id_mo + '"' )
            results = cursor.fetchone()
            usex = results[4]

            client = MongoClient('mongo', 27017)
            db = client.photo_flask
            coll = db.images
            try:
                data = coll.find({"line_id": line_id_mo, 'kinds': reco_clo})
                noteimg = random.randint(0,data.count()-1)
                with open('./images/user_clo/'+ line_id_mo + '.jpg', 'wb') as fn:
                            fn.write(data[noteimg]['imagecontent'])
            except ValueError:
                rmc = ['背心', 'T-shirt', '襯衫']
                rmc.remove(reco_clo)
                try:
                    data = coll.find({"line_id": line_id_mo, 'kinds': rmc[0]})
                    noteimg = random.randint(0,data.count()-1)
                    reco_clo = rmc[0]
                    with open('./images/user_clo/'+ line_id_mo + '.jpg', 'wb') as fn:
                                fn.write(data[noteimg]['imagecontent'])

                except ValueError:
                    rmc.remove(reco_clo)
                    data = coll.find({"line_id": line_id_mo, 'kinds': rmc[0]})
                    noteimg = random.randint(0,data.count()-1)
                    reco_clo = rmc[0]
                    with open('./images/user_clo/'+ line_id_mo + '.jpg', 'wb') as fn:
                                fn.write(data[noteimg]['imagecontent'])

            user_add1 = ImageSendMessage(original_content_url=server_url + '/images/user_clo/' + line_id_mo + '.jpg',
                                        preview_image_url=server_url + '/images/user_clo/' + line_id_mo + '.jpg')

            uss = {'男': 'M', '女': 'F'}
            r = redis.Redis(host='redis', port=6379, decode_responses=True)
            rk = r.keys("*_"+reco_clo+"_"+uss[usex]+"*")
            randomlist = random.sample(rk, 5)
            rn, ri, ru = [], [], []
            for clora in randomlist:
                rn.append(r.get(clora).split('___')[0])
                ri.append(r.get(clora).split('___')[1])
                ru.append(r.get(clora).split('___')[2])
            Carousel_templateC = TemplateSendMessage(
                    alt_text='Carousel template',
                    template=CarouselTemplate(
                    columns=[
                        CarouselColumn(
                            thumbnail_image_url=ri[0],
                            title=rn[0],
                            text=reco_clo,
                            actions=[
                                URITemplateAction(
                                    label='前往選購',
                                    uri=ru[0]
                                )
                            ]
                        ),
                        CarouselColumn(
                            thumbnail_image_url=ri[1],
                            title=rn[1],
                            text=reco_clo,
                            actions=[
                                URITemplateAction(
                                    label='前往選購',
                                    uri=ru[1]
                                )
                            ]
                        ),
                        CarouselColumn(
                            thumbnail_image_url=ri[2],
                            title=rn[2],
                            text=reco_clo,
                            actions=[
                                URITemplateAction(
                                    label='前往選購',
                                    uri=ru[2]
                                )
                            ]
                        ),
                        CarouselColumn(
                            thumbnail_image_url=ri[3],
                            title=rn[3],
                            text=reco_clo,
                            actions=[
                                URITemplateAction(
                                    label='前往選購',
                                    uri=ru[3]
                                )
                            ]
                        ),
                        CarouselColumn(
                            thumbnail_image_url=ri[4],
                            title=rn[4],
                            text=reco_clo,
                            actions=[
                                URITemplateAction(
                                    label='前往選購',
                                    uri=ru[4]
                                )
                            ]
                        ),
                    ]
                )
                )
            Confirm_templateT = TemplateSendMessage(
                alt_text='行程提醒',
                template=ConfirmTemplate(
                    title='行程提醒',
                    text='是否需要新增行程提醒?',
                    actions=[                              
                        DatetimePickerAction(
                                    label="新增提醒",
                                    data="usertime"+"/"+str(noteimg)+"/"+reco_clo,
                                    mode="datetime",
                                    initial=(datetime.datetime.now() + datetime.timedelta(hours=8)).strftime("%Y-%m-%dT%H:%M"),
                                    min=(datetime.datetime.now() + datetime.timedelta(hours=8)).strftime("%Y-%m-%dT%H:%M"),
                                    max="2038-09-08T23:59"
                                ),
                        MessageTemplateAction(
                            label='暫時不用',
                            text='不用謝謝',

                                )
                            ]
                        )
                    )

            user_profile = line_bot_api.get_profile(event.source.user_id)
            client = MongoClient('mongo', 27017)
            db = client.photo_flask
            coll = db.userevent
            data = coll.find({"line_id": vars(user_profile)["user_id"]}).sort('_id',-1).limit(1)

            weather_input = date_func(data[0]['event'])
#             print(weather_input)
            try:
                weather_output = return_weather(weather_input)
#                 print(weather_output.split(' '))
                weather_final = "桃園市"+"\n"+weather_output.split('*')[0] + weather_output.split('*')[1] +"\n天氣狀態: " \
                                    + weather_output.split('*')[2] +"\n"+ weather_output.split('*')[3] \
                                    +"\n氣溫: "+ weather_output.split('*')[5]+"～" + weather_output.split('*')[4]
#                     print(weather_final)
                line_bot_api.reply_message(event.reply_token, [TextSendMessage(text=weather_final), TextSendMessage(text='推薦您穿著：'+reco_clo), user_add1, Carousel_templateC, Confirm_templateT])
            except:
                line_bot_api.reply_message(event.reply_token, [TextSendMessage(text='推薦您穿著：'+reco_clo), user_add1, Carousel_templateC, Confirm_templateT])

        except TypeError:
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text='沒有使用者資料，請先加入會員哦～'))
        except ValueError:
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text='沒有相關衣服哦～'))

    elif msg == "進行正式相關的活動，開始推薦":
        import pymysql
        import random
        import redis
        try:
            db = pymysql.connect("mysql","root",secretFileContentJson["mysql_pass"],"user_data" )
            cursor = db.cursor()
            cursor.execute('select * from line_user where line_id = '+ '"' + line_id_mo + '"' )
            results = cursor.fetchone()
            usex = results[4]

            closs = {'男':'西裝外套', '女':'洋裝'}
            reco_clo = closs[usex]
            client = MongoClient('mongo', 27017)
            db = client.photo_flask
            coll = db.images
            data = coll.find({"line_id": line_id_mo, 'kinds': reco_clo})
            noteimg = random.randint(0,data.count()-1)
            with open('./images/user_clo/'+ line_id_mo + '.jpg', 'wb') as fn:
                        fn.write(data[noteimg]['imagecontent'])
            user_add1 = ImageSendMessage(original_content_url=server_url + '/images/user_clo/' + line_id_mo + '.jpg',
                                        preview_image_url=server_url + '/images/user_clo/' + line_id_mo + '.jpg')

            uss = {'男': 'M', '女': 'F'}
            r = redis.Redis(host='redis', port=6379, decode_responses=True)
            rk = r.keys("*"+reco_clo+"*")
            randomlist = random.sample(rk, 5)
            rn, ri, ru = [], [], []
            for clora in randomlist:
                rn.append(r.get(clora).split('___')[0])
                ri.append(r.get(clora).split('___')[1])
                ru.append(r.get(clora).split('___')[2])
            Carousel_templateC = TemplateSendMessage(
                    alt_text='Carousel template',
                    template=CarouselTemplate(
                    columns=[
                        CarouselColumn(
                            thumbnail_image_url=str(ri[0]).replace(' ','%20').replace('(','%28').replace(')','%29'),
                            title=str(rn[0]),
                            text=reco_clo,
                            actions=[
                                URITemplateAction(
                                    label='前往選購',
                                    uri=str(ru[0])
                                )
                            ]
                        ),
                        CarouselColumn(
                            thumbnail_image_url=str(ri[1]).replace(' ','%20').replace('(','%28').replace(')','%29'),
                            title=str(rn[1]),
                            text=reco_clo,
                            actions=[
                                URITemplateAction(
                                    label='前往選購',
                                    uri=str(ru[1])
                                )
                            ]
                        ),
                        CarouselColumn(
                            thumbnail_image_url=str(ri[2]).replace(' ','%20').replace('(','%28').replace(')','%29'),
                            title=str(rn[2]),
                            text=reco_clo,
                            actions=[
                                URITemplateAction(
                                    label='前往選購',
                                    uri=str(ru[2])
                                )
                            ]
                        ),
                        CarouselColumn(
                            thumbnail_image_url=str(ri[3]).replace(' ','%20').replace('(','%28').replace(')','%29'),
                            title=str(rn[3]),
                            text=reco_clo,
                            actions=[
                                URITemplateAction(
                                    label='前往選購',
                                    uri=str(ru[3])
                                )
                            ]
                        ),
                        CarouselColumn(
                            thumbnail_image_url=str(ri[4]).replace(' ','%20').replace('(','%28').replace(')','%29'),
                            title=str(rn[4]),
                            text=reco_clo,
                            actions=[
                                URITemplateAction(
                                    label='前往選購',
                                    uri=str(ru[4])
                                )
                            ]
                        ),
                    ]
                )
                )
            Confirm_templateT = TemplateSendMessage(
                alt_text='行程提醒',
                template=ConfirmTemplate(
                    title='行程提醒',
                    text='是否需要新增行程提醒?',
                    actions=[                              
                        DatetimePickerAction(
                                    label="新增提醒",
                                    data="usertime"+"/"+str(noteimg)+"/"+reco_clo,
                                    mode="datetime",
                                    initial=(datetime.datetime.now() + datetime.timedelta(hours=8)).strftime("%Y-%m-%dT%H:%M"),
                                    min=(datetime.datetime.now() + datetime.timedelta(hours=8)).strftime("%Y-%m-%dT%H:%M"),
                                    max="2038-09-08T23:59"
                                ),
                        MessageTemplateAction(
                            label='暫時不用',
                            text='不用謝謝',

                                )
                            ]
                        )
                    )

            user_profile = line_bot_api.get_profile(event.source.user_id)
            client = MongoClient('mongo', 27017)
            db = client.photo_flask
            coll = db.userevent
            data = coll.find({"line_id": vars(user_profile)["user_id"]}).sort('_id',-1).limit(1)

            weather_input = date_func(data[0]['event'])
            try:
                weather_output = return_weather(weather_input)
                weather_final = "桃園市"+"\n"+weather_output.split('*')[0] + weather_output.split('*')[1] +"\n天氣狀態: " \
                                    + weather_output.split('*')[2] +"\n"+ weather_output.split('*')[3] \
                                    +"\n氣溫: "+ weather_output.split('*')[5]+"～" + weather_output.split('*')[4]
#                     print(weather_final)
                line_bot_api.reply_message(event.reply_token, [TextSendMessage(text=weather_final), TextSendMessage(text='推薦您穿著：'+reco_clo), user_add1, Carousel_templateC, Confirm_templateT])
            except:
                line_bot_api.reply_message(event.reply_token, [TextSendMessage(text='推薦您穿著：'+reco_clo), user_add1, Carousel_templateC, Confirm_templateT])

        except TypeError:
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text='沒有使用者資料，請先加入會員哦～'))
        except ValueError:
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text='沒有相關衣服哦～'))

    elif msg =='不是運動相關的活動，請換選項' or msg =='不是休閒相關的活動，請換選項' or msg =='不是正式相關的活動，請換選項':
        pass
    elif msg =='不是西裝外套，請換選項' or msg =='不是襯衫，請換選項' or msg =='不是T-shirt，請換選項' or msg =='不是洋裝，請換選項' or msg =='不是背心，請換選項':
        pass     
    elif msg.isdigit() == False:
        reverse = {0:"運動", 1:"休閒", 2:"正式"}
        msgcut = word(msg)
        clf3 = joblib.load('./module/456.pkl')
        vec = joblib.load('./module/123.pkl')
        msgvec = vec.transform([msgcut])
        print(reverse[clf3.predict(msgvec)[0]])
        usev = reverse[clf3.predict(msgvec)[0]]
        Confirm_template = TemplateSendMessage(
        alt_text='確認情境類型',
        template=ConfirmTemplate(
            title='確認情境類型',
            text='請問您是否是要進行'+ usev + '相關的活動呢?',
            actions=[                              
                PostbackTemplateAction(
                    label='是的',
                    text='進行'+usev+'相關的活動，開始推薦',
                    data="Y" + "/" + msg+"/"+usev
                ),
                PostbackTemplateAction(
                    label='不是',
                    text='不是'+ usev +'相關的活動，請換選項',
                    data="No" + "/" + msg+"/"+usev

                )
                ]
                )
                )
        line_bot_api.reply_message(event.reply_token,Confirm_template)

        client = MongoClient('mongo', 27017)
        db = client.photo_flask
        image_collection = db.userevent 
        db2 = client.es_userevent
        image2_collection = db2.userevent
        
        dic = {"line_id": vars(user_profile)["user_id"],
                "kinds": usev,
               "up_date": datetime.datetime.now() + datetime.timedelta(hours=8),
               "event": msg
               }
        image_collection.insert(dic)
        dic = {"line_id": vars(user_profile)["user_id"],
                "kinds": usev,
               "up_date": datetime.datetime.now(),
               }
        image2_collection.insert(dic)

    else:
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="抱歉我還在學習中＠＠"))

/opt/conda/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [5]:
# 使用者或是chatbot自身回傳data給予chatbot做反應
@handler.add(PostbackEvent)
def handle_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    
    line_id_mo = vars(user_profile)["user_id"]
    
    usermsgevent = event.postback.data
    if usermsgevent.split("/")[0] == "Yc":
        from pymongo import MongoClient
        import pymongo

        client = MongoClient('mongo', 27017)
        db = client.photo_flask
        coll = db.images
        myquery = { "line_id": usermsgevent.split("/")[1] }
        newvalues = { "$set": { "kinds": usermsgevent.split("/")[2] } }
        coll.find_one_and_update(myquery, newvalues, sort=[('_id', pymongo.DESCENDING)])
        
        db2 = client.es_images
        coll2 = db2.images
        myquery = { "line_id": usermsgevent.split("/")[1] }
        newvalues = { "$set": { "kinds": usermsgevent.split("/")[2] } }
        coll2.find_one_and_update(myquery, newvalues, sort=[('_id', pymongo.DESCENDING)])
        
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="加入一件"+ usermsgevent.split("/")[2] + "，感謝使用者"))
        
    if usermsgevent.split("/")[0] == "N":
        
        from pymongo import MongoClient
        import pymongo

        user_profile = line_bot_api.get_profile(event.source.user_id)
        line_id_mo = vars(user_profile)["user_id"]
        usermsg = usermsgevent.split("/")[2]
        userevent = usermsgevent.split("/")[4]
        
        client = MongoClient('mongo', 27017)
        db = client.photo_flask   
        image_collection = db.userevent
        db2 = client.es_userevent   
        image2_collection = db2.userevent
        
        myquery = { "line_id": vars(user_profile)["user_id"] }
        newvalues = { "$set": { "kinds": userevent } }       
        image_collection.find_one_and_update(myquery, newvalues, sort=[('_id', pymongo.DESCENDING)])
        image2_collection.find_one_and_update(myquery, newvalues, sort=[('_id', pymongo.DESCENDING)])

    elif usermsgevent.split("/")[0] == "No":
        
        eventall = ['正式', '休閒', '運動']
        eventall.remove(usermsgevent.split("/")[2])
        print(eventall)
        
        eventchange = TemplateSendMessage(
        alt_text='事件選項',
        template=ButtonsTemplate(
                title='情境選項',
                text='請選擇以下選項',
                actions=[
                    PostbackTemplateAction(
                        label=eventall[0],
                        text='進行' + eventall[0] + '相關的活動，開始推薦',
                        data="N"+"/"+usermsgevent+"/"+eventall[0]
                    ),
                    PostbackTemplateAction(
                        label=eventall[1],
                        text='進行' + eventall[1] + '相關的活動，開始推薦',
                        data="N"+"/"+usermsgevent+"/"+eventall[1]
                    )
                ]
            )
        )
        
        line_bot_api.reply_message(event.reply_token, eventchange)
        
    elif usermsgevent.split("/")[0] == "clo":
        usclo = usermsgevent.split("/")[2]
        clok2 = ["西裝外套", "襯衫", "T-shirt", "洋裝", "背心"]
        print(usclo)
        clok2.remove(usclo)
        clochange = TemplateSendMessage(
        alt_text='事件選項',
        template=ButtonsTemplate(
                title='衣服選項',
                text='請選擇以下選項',
                actions=[
                    PostbackTemplateAction(
                        label=clok2[0],
                        data="U"+"/"+usermsgevent.split("/")[1]+"/"+ clok2[0]
                    ),
                    PostbackTemplateAction(
                        label=clok2[1],
                        data="U"+"/"+usermsgevent.split("/")[1]+"/"+ clok2[1]
                    ),
                    PostbackTemplateAction(
                        label=clok2[2],
                        data="U"+"/"+usermsgevent.split("/")[1]+"/"+ clok2[2]
                    ),
                    PostbackTemplateAction(
                        label=clok2[3],
                        data="U"+"/"+usermsgevent.split("/")[1]+"/"+ clok2[3]
                    ),
                ]
            )
        )
        line_bot_api.reply_message(event.reply_token, clochange)
    
    elif usermsgevent.split("/")[0] == "U":
        from pymongo import MongoClient
        import pymongo

        client = MongoClient('mongo', 27017)
        db = client.photo_flask
        coll = db.images
        db2 = client.es_images
        coll2 = db2.images

        myquery = { "line_id": usermsgevent.split("/")[1] }
        newvalues = { "$set": { "kinds": usermsgevent.split("/")[2] } }

        coll.find_one_and_update(myquery, newvalues, sort=[('_id', pymongo.DESCENDING)])
        coll2.find_one_and_update(myquery, newvalues, sort=[('_id', pymongo.DESCENDING)])
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="加入一件"+ usermsgevent.split("/")[2] + "，感謝使用者"))
        
    elif usermsgevent.split("/")[0] == "usertime":
        from pymongo import MongoClient
        import pymongo
        import os
        user_profile = line_bot_api.get_profile(event.source.user_id)
        print(vars(user_profile)["user_id"])
        
        client = MongoClient('mongo', 27017)
        db = client.photo_flask
        coll = db.userevent
        os.mkdir("./images/userdate/" + event.postback.params['datetime'] +"_"+ vars(user_profile)["user_id"])
        
        data = coll.find({"line_id": vars(user_profile)["user_id"]}).sort('_id',-1).limit(1)
        data[0]['line_id']
        with open("./images/userdate/" + event.postback.params['datetime'] +"_"+ vars(user_profile)["user_id"] +"/"+vars(user_profile)["user_id"] +".txt", 'w', encoding = 'utf-8') as fn:
            fn.write(data[0]['line_id']+','+data[0]['kinds']+','+data[0]['event'])
            
        client = MongoClient('mongo', 27017)
        db = client.photo_flask
        coll = db.images   
        data = coll.find({"line_id": vars(user_profile)["user_id"], 'kinds': usermsgevent.split("/")[2]})
        ppp = int(usermsgevent.split("/")[1])
        
        with open("./images/userdate/" + event.postback.params['datetime'] +"_"+ vars(user_profile)["user_id"] +"/"+vars(user_profile)["user_id"] +".jpg" , 'wb') as fn:
            fn.write(data[ppp]['imagecontent'])
        
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="已加入提醒"))
        
    elif usermsgevent == "加入會員":      
            member_join = ImagemapSendMessage(
                        base_url='https://i.imgur.com/wfGkbO6.png',
                        alt_text='加入會員',
                        base_size=BaseSize(height=400, width=1040),
                        actions=[
                            URIImagemapAction(
                                link_uri='line://app/1565906077-d3vbPQPb' + "?line_id=" + line_id_mo,
                                area=ImagemapArea(
                                    x=0, y=0, height=400, width=1040
                                )
                            )
                        ]
                    )
            line_bot_api.reply_message(event.reply_token, member_join)
    elif usermsgevent == "關於我":
            information = TemplateSendMessage(
                alt_text='關於我',
                template=ButtonsTemplate(
                    title='關於我的智能衣櫃',
                    text='請選擇您想了解的內容',
                    thumbnail_image_url='https://i.imgur.com/sWqODWc.jpg',
                    actions=[
                        PostbackTemplateAction(
                            label='機器人簡介',
                            data="機器人簡介"
                        ),
                        PostbackTemplateAction(
                            label='基本使用說明',
                            data="基本使用說明"
                        ),
                        MessageTemplateAction(
                            label='功能說明',
                            text='功能說明'
                        ),
                        URITemplateAction(
                            label='關於開發者',
                            uri='https://i.imgur.com/RLpuxAr.png'
                        )
                    ]
                )
            )
            line_bot_api.reply_message(event.reply_token, information)
    elif usermsgevent == "情境輸入":
            line_bot_api.reply_message(event.reply_token, [TextSendMessage(text="請鍵入情境，系統將給予天氣資訊並自動幫您挑選服裝，另外也可以設定行程提醒哦～"), StickerSendMessage(package_id=1, sticker_id=2)])
    elif usermsgevent == "查看個人資料":
            import pymysql
            from pymongo import MongoClient
            import pymongo
            db = pymysql.connect("mysql","root",secretFileContentJson["mysql_pass"],"user_data" )
            cursor = db.cursor()
            cursor.execute('select * from line_user where line_id = '+ '"' + line_id_mo + '"' )
            results = cursor.fetchone()
            client = MongoClient('mongo', 27017)
            db = client.photo_flask
            
            coll = db.images
            a = coll.find({"line_id": line_id_mo})
            T = coll.find({"line_id": line_id_mo, 'kinds': 'T-shirt'})
            s = coll.find({"line_id": line_id_mo, 'kinds': '西裝外套'})
            h = coll.find({"line_id": line_id_mo, 'kinds': '襯衫'})
            d = coll.find({"line_id": line_id_mo, 'kinds': '洋裝'})
            v = coll.find({"line_id": line_id_mo, 'kinds': '背心'})
            ecol = db.userevent
            aa = ecol.find({"line_id": line_id_mo})
            ss = ecol.find({"line_id": line_id_mo, 'kinds': '運動'})
            pp = ecol.find({"line_id": line_id_mo, 'kinds': '正式'})
            rr = ecol.find({"line_id": line_id_mo, 'kinds': '休閒'})
            
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text ="會員暱稱: " + results[1] + "\n" 
                                + "信箱: " + results[2] + "\n" + "手機號碼: " + results[3] + "\n" 
                                + "性別: " + results[4] + "\n" + "年齡: " + results[5] + "歲" + "\n" 
                                + "身高: " + results[6] + "公分" + "\n" + "體重: " + results[7] + "公斤" + '\n'
                                +'--------------------\n'
                                + '衣服總數: ' + str(a.count()) + '件\n' + 'T-shirt總數: ' + str(T.count()) + '件\n'
                                + '西裝外套總數: ' + str(s.count()) + '件\n' + '襯衫總數: ' + str(h.count()) + '件\n'
                                + '洋裝總數: ' + str(d.count()) + '件\n' + '背心總數: ' + str(v.count()) + '件\n'
                                +'--------------------\n'
                                + '輸入情境總次數: ' + str(aa.count()) + '次\n' + '運動情境次數: ' + str(ss.count()) + '次\n'
                                + '正式情境次數: ' + str(pp.count()) + '次\n' + '休閒情境次數: ' + str(rr.count()) + '次'                                                                  
                                ))
    elif usermsgevent == "開啟衣櫃":
            import os
            from pymongo import MongoClient
            import shutil
            client = MongoClient('mongo', 27017)
            db = client.photo_flask
            coll = db.images
            clo = []
            data = coll.find({"line_id": line_id_mo})
            if not os.path.isdir('./images/userclobox/'+ line_id_mo):
                os.mkdir('./images/userclobox/'+ line_id_mo)
            elif os.path.isdir('./images/userclobox/'+ line_id_mo):
                shutil.rmtree('./images/userclobox/'+ line_id_mo)
                os.mkdir('./images/userclobox/'+ line_id_mo)
            ppp = 1
            for i in data:
                with open('./images/userclobox/'+ line_id_mo + '/' + i['line_id'] + '$' + i['kinds'] + '$' +str(ppp)+'.jpg', 'wb') as fn:
                    fn.write(i['imagecontent'])
                ppp += 1

            user_closet = ImagemapSendMessage(
                    base_url='https://i.imgur.com/bR6cxpW.png',
                    alt_text='開啟衣櫃',
                    base_size=BaseSize(height=400, width=1040),
                    actions=[
                        URIImagemapAction(
                            link_uri = "line://app/1565906077-b6aB5w5B" + "?name=" + line_id_mo,
                            area=ImagemapArea(
                                x=0, y=0, height=400, width=1040
                            )
                        )
                    ]
                )
            line_bot_api.reply_message(event.reply_token, user_closet)   

    elif usermsgevent == "我的明星臉":

        user_faceid = ImagemapSendMessage(
                    base_url='https://i.imgur.com/8kXiEyc.png',
                    alt_text='我的明星臉',
                    base_size=BaseSize(height=400, width=1040),
                    actions=[
                        URIImagemapAction(
                            link_uri=server_url + "/face" + "?openExternalBrowser=1"  + "&name=" + line_id_mo,
                            area=ImagemapArea(
                                x=0, y=0, height=400, width=1040
                            )
                        )
                    ]
                )
        line_bot_api.reply_message(event.reply_token, user_faceid)

    elif usermsgevent == "機器人簡介":
        qrcode = ImageSendMessage(original_content_url='https://i.imgur.com/ty6a4uc.png',
                                        preview_image_url='https://i.imgur.com/ty6a4uc.png')
        botint = TextSendMessage(text="我的智能衣櫃主要功能為：\n－－－－－－－－－－\n．分析使用者上傳之服飾並建檔放入衣櫃"+
              "\n－－－－－－－－－－\n．分析使用者輸入之情境後推薦衣櫃內適合之衣物，並推播最新購物資訊，"+
              "也可依使用者需求新增行程提醒\n－－－－－－－－－－\n．由使用者上傳之照片進行明星臉辨識，並推薦相關服裝，歡迎使用者您的使用！")
        line_bot_api.reply_message(event.reply_token, [qrcode, botint])
    elif usermsgevent == "基本使用說明":
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="請點選螢幕下方之「點我開啟智能衣櫃」來開啟功能選單以使用個人智能衣櫃，也可以直接鍵入情境，系統將會自動幫您做服裝推薦及購物推薦哦～"))
    elif usermsgevent == "加入會員說明":
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="點選「加入會員」以加入智能衣櫃之會員，成功註冊後系統將會以獲得的數據進行更精確的推薦及分析。"))
    elif usermsgevent == "情境輸入說明":
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="點選「情境輸入」以使用情境判斷功能，輸入情境後系統將給予天氣資訊並自動幫您挑選服裝，確定後也可以設定行程提醒及獲得購物資訊等等的功能哦！"))
    elif usermsgevent == "拍照／圖庫上傳說明":
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="點選「拍照上傳」即可使用裝置上的相機進行拍照並上傳服飾，點選「圖庫上傳」則會開啟裝置上的相簿，在圖片上傳完畢後系統將會自動判斷服飾種類並放進您的個人衣櫃裡。"))
    elif usermsgevent == "查看個人資料說明":
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="點選「查看個人資料」即可查看你先前註冊會員時填入之資料。"))
    elif usermsgevent == "開啟衣櫃說明":
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="點選「開啟衣櫃」即可線上查看使用至今的個人衣櫃。"))
    elif usermsgevent == "我的明星臉說明":
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="點選「我的明星臉」後即可使用明星臉比對功能，您可以以拍照或圖庫上傳的方式上傳您的照片，系統將會比對多位明星並找出與您最相似的明星，並提供相關的購物資訊。"))

In [6]:
# 開啟加入會員html
import pymysql
@app.route('/userfiles', methods=['GET', 'POST'])
def hello():
    line_id = request.values.get('line_id')
    return render_template('index_1.html', line_id=line_id)

In [7]:
# 使用者在加入會員html按下送出後的動作
@app.route('/sucess', methods=['GET', 'POST'])
def sucesss():
    db = pymysql.connect("mysql", "root", secretFileContentJson["mysql_pass"], "user_data")
    cursor = db.cursor()
    
    line_id = request.values.get('line_id')
    name = request.values.get('name')
    email = request.values.get('email')
    sex = request.values.get('sex')
    phone = request.values.get('phone')
    age = request.values.get('age')
    height = request.values.get('height')
    weight = request.values.get('weight')
    emit = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
    print(name)
    
    linkRichMenuId="richmenu-02e98659ec5f8ebd88f0db2bed101d69"
    linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (line_id, linkRichMenuId)
    linkMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}
    lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
    
    try:
        if name != None:
            cursor.execute('INSERT INTO line_user (line_id,username,email,cellphone,sex,age,height,weight,time) '
                       'VALUES ("' + line_id + '","' + name + '","' + email + '","' + phone + '","' + sex + '","' + age + '","' + height + '","' + weight + '","' + emit + '");')
            db.commit()
            return render_template('register_sucess.html')
    except pymysql.err.IntegrityError:
        return render_template('register_duplicated.html')

In [8]:
# 使用者看衣櫃資訊時的網頁
@app.route('/userdb', methods=['GET', 'POST'])
def hello_world():
    client = MongoClient('mongo', 27017)
    db = client.photo_flask
    coll = db.images
    kind = request.values.get('kind')
    name = request.values.get('name')
#     print(kind)
#     print(name)
    clo = []
    if kind == None:
        clo = []
    if kind == "all":
        imgp = os.listdir('./images/userclobox/'+ name)
        for n in imgp:
            if n != '.ipynb_checkpoints':
                clo_dict = {'name': './images/userclobox/'+name+'/'+n ,
                            'author': n.split('$')[1]}
                clo.append(clo_dict)               
    else:
        imgp = os.listdir('./images/userclobox/'+ name)
        for n in imgp:
            if n != '.ipynb_checkpoints':
                if kind == n.split('$')[1]:
                    clo_dict = {'name': './images/userclobox/'+name+'/'+ n ,
                                'author': n.split('$')[1]}
                    clo.append(clo_dict)
    return render_template('index.html', books=clo, name=name)

In [ ]:
'''
若收到圖片消息時，
先回覆用戶文字消息，並從Line上將照片拿回。
'''
import random
import time
from confluent_kafka import Producer
import base64
@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)

    
    message_content = line_bot_api.get_message_content(event.message.id)

    # 模型加入的地方
#     clok = {0: "T-shirt", 1:"西裝外套", 2:"襯衫", 3:"洋裝", 4:"背心"} 
#     rd = random.randint(0,4)
#     clokk = clok[rd]
    
    client = MongoClient('mongo', 27017)
    db = client.photo_flask
    image_collection = db.images
    db2 = client.es_images
    image2_collection = db2.images
    for chunk in message_content.iter_content(chunk_size=1024*1024):            
        dic = {"line_id": vars(user_profile)["user_id"],
               "kinds": '未確認',
               "up_date": datetime.datetime.now() + datetime.timedelta(hours=8)
               }

        dic["imagecontent"] = binary.Binary(chunk)
        image_collection.insert_one(dic)
        
        dic = {"line_id": vars(user_profile)["user_id"],
               "kinds": '未確認',
               "up_date": datetime.datetime.now()
               }
        image2_collection.insert_one(dic)
        
        props = {'bootstrap.servers': 'kafka4Br1:9092'}
        producer = Producer(props)
        topicName = 'onionTopic1'
        producer.produce(topicName, base64.b64encode(binary.Binary(chunk)), vars(user_profile)["user_id"])
        producer.flush()


In [ ]:
# 明星臉上傳載入之網頁及資料庫
import flask
from bson import binary
from flask import render_template
import base64
import pymongo
import bson.binary
from gridfs import *
from pymongo import MongoClient
from io import StringIO
import os
from pathlib import Path
import random
import zlib

@app.route('/upload', methods=['POST'])
def upload():
    line_id = request.values.get('line_id')
    compress = request.values.get('uploaded_file')
#     print(compress)
#     f = flask.request.files['uploaded_file']
    decoded = base64.decodebytes(compress.split(',')[1].encode("ascii"))

    client = MongoClient('mongo', 27017)
    db = client.photo_flask
    image_collection = db.face
    dic = {"line_id": line_id}
    content = decoded
    dic["imagecontent"] = content
    image_collection.insert(dic)
    
#     os.remove('./images/userface/'+ line_id + '.jpg')
    
    with open('./images/userface/'+ line_id + '.jpg', 'wb') as fn:
        fn.write(decoded)
 
    return flask.redirect('/result?userid='+line_id)

@app.route('/result')
def index1():
    userid = request.values.get('userid')
    
    random_male = random.choice(os.listdir("./images/male"))
    random_female = random.choice(os.listdir("./images/female"))
    male_similarity = random.randint(60,90)
    female_similarity = random.randint(60,90)
    
    cloth_dic = {'張鈞甯.jpg':'https://shopee.tw/%E7%84%A1%E8%A2%96%E6%B4%8B%E8%A3%9D%E2%9C%A8%E5%AF%A6%E6%8B%8D%E5%A4%8F%E5%AD%A3%E6%96%B0%E6%AC%BE%E6%AD%90%E7%BE%8E%E8%B5%AB%E6%9C%AC%E5%B0%8F%E9%A6%99%E9%A2%A8%E6%80%A7%E6%84%9F%E9%9C%B2%E8%83%8C%E7%B3%BB%E5%B8%B6%E8%9D%B4%E8%9D%B6%E7%B5%90%E5%A5%B3%E7%94%9F%E4%BC%91%E9%96%92%E5%A4%96%E5%87%BA%E6%A0%BC%E5%AD%90%E5%90%8A%E5%B8%B6%E9%80%A3%E8%A1%A3%E8%A3%99%E5%AD%90-i.88486235.1934230936?fbclid=IwAR1eaiL8LwQYz2CVYAyE5eThgmsDJHhlsYeiKrvezXJQGfnhp2VF6LzSJDU',
             '林依晨.jpg':'https://tw.mall.yahoo.com/item/%E3%80%8AAB8539%E3%80%8B%E5%81%87%E5%85%A9%E4%BB%B6%E5%BC%8F%E9%BB%9E%E9%BB%9E%E9%85%8D%E8%89%B2%E5%8E%9A%E9%9B%AA%E7%B4%A1%E4%B8%8A%E8%A1%A3-%E8%A5%AF%E8%A1%AB-OrangeBe-p0907174237327',
             '桂綸鎂.jpg':'https://www.miu-star.com.tw/SalePage/Index/5268535?gclid=CjwKCAjw_MnmBRAoEiwAPRRWW0LBdWps886BSyShW1y3NhPVDQXEnGjTS1QjGAz07wa8AJVmFFZWTBoCSMQQAvD_BwE',
             '陳意涵.jpg':'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=6186367&osm=Ad07&utm_source=googleshop&utm_medium=googleshop_USC&utm_content=bn&gclid=CjwKCAjw_MnmBRAoEiwAPRRWW-LBNjq5WGv6dT6Esp6NXgGrMsHz3W83LK7zC21VgEYMBeOj4BB4ehoCDXYQAvD_BwE&fbclid=IwAR3X4hiy8wsbHv-yu9OsFZ7VHwOrRj_WDNgiGWyPfKIdbTr47AKk9CIG1tc',
             '周杰倫.jpg':'https://www.momoshop.com.tw/category/DgrpCategory.jsp?d_code=1307602414&mdiv=1306700000-bt_2_118_01-bt_2_118_01_e1&ctype=B&p_orderType=1',
             '林志穎.jpg':'https://www.101.com.tw/goods-details-4276.html?utm_term=41439&gclid=CjwKCAjw_MnmBRAoEiwAPRRWW4V5wBnH-1-a18k6jvNSKGVC-RwuFUq0Ft6TacARzy_93enXjKYfxhoCpG4QAvD_BwE',
             '林書豪.jpg':'https://www.myprotein.tw/sports-t-shirts/seamless-short-sleeve-t-shirt/11429309.html?affil=thggpsad&switchcurrency=TWD&shippingcountry=TW&variation=11426602&thg_ppc_campaign=71700000048921140&product_id=11426602&gclid=CjwKCAjw_MnmBRAoEiwAPRRWW_K-BdThsUi1x7x75w3wgMBJrUsxYLfDu2QZTJ4Y51V6lmEXiCPsSxoCgukQAvD_BwE&gclsrc=aw.ds',
             '陳柏霖.jpg':'https://www.zalora.com.tw/polo-ralph-lauren-polo-%E7%B4%85%E8%89%B2-686587.html'
             }

    website_male= cloth_dic[random_male]
    website_female= cloth_dic[random_female]
    
    face = './images/userface/'+ userid + '.jpg'
#     print(face)
    
    return render_template('index_uploadcomplete.html', random_male = random_male, random_female = random_female, website_male = website_male, website_female = website_female, face = face, male_similarity = male_similarity, female_similarity = female_similarity)

@app.route('/face', methods=['GET', 'POST'])
def index2():
    line_id = request.values.get('name')
#     print(line_id)
    return render_template('index_upload.html', line_id = line_id)

In [ ]:
'''
啟動Server
'''
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:233: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
172.27.0.11 - - [26/May/2019 16:49:41] "POST / HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 16:49:42] "POST / HTTP/1.1" 200 -
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.739 seconds.
Prefix dict has been built succesfully.
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.20.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.20.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserW

運動


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:594: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:599: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
172.27.0.11 - - [26/May/2019 16:50:45] "POST / HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 16:50:48] "POST / HTTP/1.1" 200 -
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:116: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
172.27.0.11 - - [26/May/2019 16:50:50] "POST / HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 16:50:52] "GET /images/user_clo/U8b7ecef1297d207a53212ca370a703f0.jpg HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 16:50:53] "GET /images/user_clo/U8b7ecef1297d207a53212ca370a703f0.jpg HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 16:52:11] "GET /images/user_clo/U8b7ecef1297d207a53212ca370a703f0.jpg HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2

西裝外套


172.27.0.11 - - [26/May/2019 16:52:27] "POST / HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 16:52:27] "GET /images/user_clo/U8b7ecef1297d207a53212ca370a703f0.jpg HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 16:52:30] "POST / HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 16:52:31] "GET /images/user_clo/U8b7ecef1297d207a53212ca370a703f0.jpg HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 17:40:38] "POST / HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 17:40:39] "POST / HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 17:40:51] "POST / HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 17:40:52] "POST / HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 17:40:55] "POST / HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 17:40:57] "POST / HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 17:41:01] "GET /userdb?name=U8b7ecef1297d207a53212ca370a703f0 HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 17:41:03] "GET /favicon.ico HTTP/1.1" 404 -
172.27.0.11 - - [26/May/2019 17:41:37] "POST / HTTP/1.1" 200 -
172.27.0.11 - - [26/May

運動


172.27.0.11 - - [26/May/2019 17:43:59] "POST / HTTP/1.1" 200 -


['正式', '休閒']


172.27.0.11 - - [26/May/2019 17:44:01] "POST / HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 17:44:03] "POST / HTTP/1.1" 200 -
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:268: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
172.27.0.11 - - [26/May/2019 17:44:05] "POST / HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 17:44:07] "GET /images/user_clo/U8b7ecef1297d207a53212ca370a703f0.jpg HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 17:44:07] "GET /images/user_clo/U8b7ecef1297d207a53212ca370a703f0.jpg HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 17:44:11] "GET /images/user_clo/U8b7ecef1297d207a53212ca370a703f0.jpg HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 17:44:12] "POST / HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 17:44:27] "GET /images/user_clo/U8b7ecef1297d207a53212ca370a703f0.jpg HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 17:44:28] "POST / HTTP/1.1" 200 -
172.27.0.11 - - [26/May/2019 17:44:30] "POST / HTTP/1.1" 200 -
172.27.0.11 - -